# CSC478 Final Project
# Jonggoo Kang
# Predict Stock Prices using Twitter API data 

For the project, I collected Twitter streaming API from 9:00am to 4:00pm based on New York time for a week (May22 to May26, and Jun 5th) about $APPL. I believe that emotions on SNS would affect the stock market in some ways. Therefore, I tried to predict a stock prices with the twitter emotions about Apple.

First of all, I used a classification model which discriminates between positive, neutral, and negative tweets. I also utilized a time series of daily returns based on the closing prices. Finally, I completed the analysis using Logistic Regression to check if Twitter sentiment and daily stock prices are related. 

### 1. Collecting Twitter Streaming API related to '$APPL'


First of all, you need to have your own Twitter account. 
By using "tweepy" module, you can link you account and get the API from Twitter.

In [1]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'dG5YKZA7UUzj8zASL6HqJ8S5E'
consumer_secret = 'Qx70JFu8cbx8fciZcnZ9RWTkGNA8QAHrQFf9ovh3JpqGNuk4cz'
access_token = '854203928636006400-MFsbyBvbpCToPgh8fgBgWkTxqtQ5ErN'
access_secret = 'PpITeoGd3tmQcc8EX1wWPsxtymdjYnm1hzIN2gKTiTw7g'

 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [2]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import os
os.chdir('/Users/jaygkay/Desktop/')

class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('tweetsAPI.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['chicago']) #$AAPL

KeyboardInterrupt: 

### 2. Manipulating the API


In [3]:
import os
os.chdir('/Users/jaygkay/Desktop/jaygkay/project')
import pandas as pd
import numpy as np
import json

readfile() is to read 'json' file 

In [4]:
def readfile(file):
    
    tweets_data = []
    tweets_file = open(file, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
    return tweets_data

tweets() gives you only text from the json file

In [5]:
def tweets(file, readfile):
    tweets_data = readfile(file)
    
    tweets = pd.DataFrame()
    tweets['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
    tweets['username'] = list(map(lambda tweet: tweet['user']['name'] if 'user' in tweet else None, tweets_data))
    tweets['location'] = list(map(lambda tweet: tweet['user']['location'] if 'user' in tweet else None, tweets_data))
    tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
    #return tweets

    import datetime
    created = [i for i in tweets['created_at']]
    tweetext = [j for j in tweets['text']]
    time = []
    text = []
    for i in range(len(created)):
        #10 to 3 based on New York time (14 ~ 19)
        #12pm to 4pm () based on New york///// chicago 11am-3pm & LA 9am ~ 1pm
        if created[i].split()[3] >= '13:00:00' and created[i].split()[3] < '19:00:00' :
            time.append(created[i])
            text.append(tweetext[i])
    df = pd.DataFrame()
    df['time'] = time
    df['text'] = text
    df.shape, df.head()
    
    textonly = df['text']
    puretext = textonly.str.lower()

    return puretext

you can get rid of (@username, URL, Emoji, and special characters) that ruins sentiment analysis with "cleantext()". Also, "cleanit()" gives you only text filtered by cleantext()

In [6]:
from textblob import TextBlob
import re
def cleantext(file):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(rt)", " ", file).split())
    
def cleanit(file, tweets):
    a = []
    puretext = tweets(file, readfile)
    for i in puretext:
        a.append(cleantext(i))
    return a   




sentiment() uses "TextBlob" module that evaluate a sentence in positive, neutral, or negative emorions.

In [7]:
def sentiment(text):
        analysis = TextBlob(cleantext(text))
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

thus() gives you total numbers of positive, neutral, negative emotions form the twitter data

In [8]:
def thus(file, cleanit, tweets):
    a = cleanit(file, tweets)
    sent = []
    
    for i in a:
        sent.append(sentiment(i))
    #print(['pos', 'neu', 'neg'])
    b = [sent.count('positive'), sent.count('neutral'),  sent.count('negative')]
    
    return b
   

### 3. Examples of the functions above

In [9]:
readfile('May22.json')

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Mon May 22 12:58:35 +0000 2017',
  'entities': {'hashtags': [],
   'symbols': [{'indices': [55, 60], 'text': 'AAPL'},
    {'indices': [61, 66], 'text': 'ABBV'},
    {'indices': [67, 72], 'text': 'AMGN'},
    {'indices': [73, 76], 'text': 'BA'},
    {'indices': [77, 81], 'text': 'BAC'},
    {'indices': [82, 88], 'text': 'CMCSA'},
    {'indices': [89, 94], 'text': 'CSCO'},
    {'indices': [95, 99], 'text': 'CVX'},
    {'indices': [100, 103], 'text': 'GE'},
    {'indices': [104, 107], 'text': 'HD'},
    {'indices': [108, 112], 'text': 'IBM'},
    {'indices': [113, 118], 'text': 'INTC'},
    {'indices': [119, 123], 'text': 'JNJ'},
    {'indices': [124, 128], 'text': 'JPM'},
    {'indices': [129, 133], 'text': 'KHC'},
    {'indices': [134, 137], 'text': 'KO'}],
   'urls': [{'display_url': 'seekingalpha.com/article/407396…',
     'expanded_url': 'https://seekingalpha.com/article/4073969-great-dividend-etf-simulate?source=tweet',

In [10]:
tweets('May22.json', readfile)

0      premarket most active stocks: $rdus, $tvix, $n...
1      wal-mart's huge e-commerce bet on marc lore st...
2      real time trade alerts \n\nhttps://t.co/rtpsg1...
3      $spy open trades on watch -\n\nhttps://t.co/mo...
4      rt @reddogt3: $aapl quick glance https://t.co/...
5      rt @justinpulitzer: weekend review video 5/21:...
6      rt @pelias01: great dividend etf to simulate h...
7      #trading us upgrades-downgrades $aapl $azo @br...
8      https://t.co/yabwvlztnt &lt;-- choose freedom ...
9      rt @investing_books: systematic trading by rob...
10     macd turns bearish for $aapl $twtr $crm check ...
11     benzinga: wal-mart's huge e-commerce bet on ma...
12     royal bank of canada raises apple inc. $aapl p...
13     royal bank of canada raises apple inc. $aapl p...
14     rt @benzinga: wal-mart's huge e-commerce bet o...
15     $aapl apple could top $1 trillion in market ca...
16     smart money selling $dis $aapl $fb $amzn $nflx...
17     apple could top $1 trill

In [11]:
cleanit('May22.json', tweets)

['premarket most active stocks rdus tvix nvda xiv uslv glyc oclr momo qqq aapl',
 'wal ma s huge e commerce bet on marc lore sta s paying off aapl amzn wmt',
 'real time trade ale s spy iwm gld vix uvxy gld uso fb aapl nflx tsla pcln bbry gdx amzn baba',
 'spy open trades on watch momo f rdus amgn fb aapl nflx googl uvxy iwm',
 'aapl quick glance',
 'weekend review video 5 21 vix tlt gld slv spy iwm qqq aapl amzn fb nflx goog adsk tsla nvda snap x ht',
 'great dividend etf to simulate aapl abbv amgn ba bac cmcsa csco cvx ge hd ibm intc jnj',
 'trading us upgrades downgrades aapl azo',
 'lt choose freedom from pump and dump scams a icle on my blog gpro amba tube halo aapl',
 'books systematic trading by robe carver spy dia es f spx aapl',
 'macd turns bearish for aapl twtr crm check out the complete list of technical picks here',
 'benzinga wal ma s huge e commerce bet on marc lore sta s paying off aapl amzn wmt',
 'royal bank of canada raises apple inc aapl price target to 168 00',
 'r

In [12]:
sentiment('I am bad'), sentiment('I am fantastic')

('negative', 'positive')

###  4. Start Analysis

In [13]:
thus('May22.json', cleanit, tweets)

[156, 336, 57]

In [14]:
thus('May23.json', cleanit, tweets)

[194, 491, 46]

In [15]:
thus('May24.json', cleanit, tweets)

[155, 280, 80]

In [16]:
thus('May25.json', cleanit, tweets)

[221, 277, 47]

In [17]:
thus('May26.json', cleanit, tweets)

[188, 339, 51]

In [18]:
thus('Jun5.json', cleanit, tweets)

[695, 6067, 2511]

### 5. Apple Stock Market Prices 

Used "pandas_datareader.data" to read "Apple stock prices"
Also, this program gives you binary values of 0 (decreas compared to the price yesterday) and 1 (increases). 

In [19]:
import pandas_datareader.data as web
import datetime
start = datetime.datetime(2017,5,19)
end = datetime.datetime(2017,6,6)

apple = web.DataReader("AAPL", "google", start, end)
print(apple)

#daily reutrns
df = apple
a = df['Close']
rev = []
for i in range(len(a)):
    if a[i] > a[i-1]:
        rev.append(1) #1 if increase
    else:
        rev.append(0)
print(rev)
print('Revenue with time seriese ', rev[1:])

              Open    High     Low   Close    Volume
Date                                                
2017-05-19  153.38  153.98  152.63  153.06  26960788
2017-05-22  154.00  154.58  152.91  153.99  22966437
2017-05-23  154.90  154.90  153.31  153.80  19918871
2017-05-24  153.84  154.17  152.67  153.34  19219154
2017-05-25  153.73  154.35  153.03  153.87  19235598
2017-05-26  154.00  154.24  153.31  153.61  21927637
2017-05-30  153.42  154.43  153.33  153.67  20126851
2017-05-31  153.97  154.17  152.38  152.76  24451164
2017-06-01  153.17  153.33  152.22  153.18  16404088
2017-06-02  153.58  155.45  152.89  155.45  27770715
2017-06-05  154.34  154.45  153.46  153.93  25331662
[0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
Revenue with time seriese  [1, 0, 0, 1, 0, 1, 0, 1, 1, 0]


### 6. Logistic Regression

In [20]:
xVar = thus('May22.json', cleanit, tweets), thus('May23.json', cleanit, tweets),thus('May24.json', cleanit, tweets),thus('May25.json', cleanit, tweets), thus('May26.json', cleanit, tweets), thus('Jun5.json', cleanit, tweets)
xVar

([156, 336, 57],
 [194, 491, 46],
 [155, 280, 80],
 [221, 277, 47],
 [188, 339, 51],
 [695, 6067, 2511])

In [21]:
yVar = rev[1],rev[2],rev[3],rev[4],rev[5],rev[-1] 
yVar

(1, 0, 0, 1, 0, 0)

In [22]:
df = pd.DataFrame()
df['positive'] = xVar[0][0], xVar[1][0], xVar[2][0], xVar[3][0], xVar[4][0], xVar[5][0]
df['neutral'] = xVar[0][1], xVar[1][1], xVar[2][1], xVar[3][1], xVar[4][1], xVar[5][1]
df['negative'] = xVar[0][2], xVar[1][2], xVar[2][2], xVar[3][2], xVar[4][2],xVar[5][2],
df['target'] = yVar
df

,positive,neutral,negative,target
0,156,336,57,1
1,194,491,46,0
2,155,280,80,0
3,221,277,47,1
4,188,339,51,0
5,695,6067,2511,0


In [23]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
df.describe()

,positive,neutral,negative,target
count,6.000000,6.000000,6.000000,6.000000
mean,268.166667,1298.333333,465.333333,0.333333
std,210.583396,2337.455853,1002.246011,0.516398
min,155.000000,277.000000,46.000000,0.000000
25%,164.000000,294.000000,48.000000,0.000000
50%,191.000000,337.500000,54.000000,0.000000
75%,214.250000,453.000000,74.250000,0.750000
max,695.000000,6067.000000,2511.000000,1.000000


In [25]:
x_var = np.array(df[['positive', 'neutral', 'negative']])
y_var = np.array(df['target'])
print(x_var.shape, y_var.shape)

#Standardized
scaler = StandardScaler()
x_var = scaler.fit_transform(x_var)
print(x_var)

xTrain, xTest, yTrain, yTest = train_test_split(x_var, y_var, test_size=0.3, random_state = 10)
print("xTrain size: ", xTrain.shape)
print("xTest size: ", xTest.shape)
print("yTrain size: ", yTrain.shape)
print("yTest size: ", yTest.shape)

(6, 3) (6,)
[[-0.58348583 -0.45099605 -0.44630435]
 [-0.38581158 -0.37835553 -0.45832724]
 [-0.58868778 -0.47724036 -0.42116558]
 [-0.24535882 -0.47864631 -0.45723425]
 [-0.4170233  -0.4495901  -0.45286229]
 [ 2.22036732  2.23482835  2.23589371]]
xTrain size:  (4, 3)
xTest size:  (2, 3)
yTrain size:  (4,)
yTest size:  (2,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [26]:
model = LogisticRegression()
model.fit(xTrain, yTrain)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
print('Logistic Regression score for training set: %f' %model.score(xTrain, yTrain))
print('===========================================================================')
print("classification Report")
print(classification_report(yTest, model.predict(xTest)))

Logistic Regression score for training set: 0.500000
classification Report
             precision    recall  f1-score   support

          0       1.00      0.50      0.67         2
          1       0.00      0.00      0.00         0

avg / total       1.00      0.50      0.67         2



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
